# Additional Examples

In [1]:
# Following code is needed to preconfigure this notebook
import datetime
import sys
import os
sys.path.insert(0, os.path.abspath('../../..'))

import pyflow as pf

scratchdir = os.path.join('/', 'path', 'to', 'scratch')
filesdir = os.path.join(scratchdir, 'files')
outdir = os.path.join(scratchdir, 'out')


class CourseSuite(pf.Suite):
    """
    This CourseSuite object will be used throughout the course to provide sensible
    defaults without verbosity
    """
    def __init__(self, name, **kwargs):
        
        config = {
            'host': pf.LocalHost(),
            'files': os.path.join(filesdir, name),
            'home': outdir,
            'defstatus': pf.state.suspended
        }
        config.update(kwargs)
        
        super().__init__(name, **config)


class LabelSetter(pf.Task):
    
    def __init__(self, *args, **kwargs):
        """
        Accepts a sequence of label-value tuples
        """
        script = [
            pf.TemplateScript(
                'ecflow_client --alter=change label {{ LABEL.name }} "{{ VALUE }}" {{ LABEL.parent.fullname }}',
                LABEL=label, VALUE=value
            ) for label, value in args
        ]
        
        name = kwargs.pop('name', 'set_labels')
        super().__init__(name, script=script, **kwargs)
        

class WaitSeconds(pf.Task):
    def __init__(self, seconds, **kwargs):
        name = kwargs.pop('name', 'wait_{}'.format(seconds))
        super().__init__(name, script='sleep {}'.format(seconds), **kwargs)

## Suite Example

In this section we construct a component of a test suite which will obtain testing data from MARS, perform some "test" on it, and then clean up after itself. This demonstrates a number of characteristics of object-oriented suite design:

1. Functionality that is configurable on a data description.
2. Functionality that is encapsulated in re-usable subcomponents
3. Delegation or inheritance to fine-tune behaviour within an existing framework

Firstly we create a helper class that can understand MARS requests, and output them in a useful format.

In [2]:
class MarsRequest:
    
    separator = ",\n    "
    
    def __init__(self, verb, request_dict):
        self._verb = verb
        self._request_dict = request_dict
        
    def __str__(self):
        return (
            self._verb +
            self.separator +            
            self.separator.join("{}={}".format(k, self._resolve(v)) for k, v in self._request_dict.items())
        )
        
    @staticmethod
    def _resolve(v):
        '''Convert values into something understood by MARS'''
        if isinstance(v, bool):
            return "on" if v else "off"
        if isinstance(v, list):
            return '/'.join(MarsRequest._resolve(vv) for vv in v)
        if isinstance(v, str) and ('/' in v or '$' in v):
            return '"{}"'.format(v)
        return str(v)

These requests are useful in the context of a `MarsTask`. This makes use of the `MarsRequest` object defined above to do something in the current working directory. It also creates a label for monitoring in ecflow and a timers file for diagnostics according to the environment variables understood by MARS.

In [3]:
mars_task_script = """
req=$(mktemp req.XXXX)
cat > $req <<@
{{ REQUEST }}
@
mars $req
rm $req
"""


class MarsTask(pf.Task):
    
    verb = None
    
    def __init__(self, request_dict, **kwargs):
        
        # Construct a MarsRequest object from the dictionary supplied
        assert self.verb is not None
        request = MarsRequest(self.verb, request_dict)
        
        name = kwargs.get('name', "{}_data".format(self.verb))
        
        super().__init__(name,
                         labels={'info': ''},
                         script=pf.TemplateScript(mars_task_script, REQUEST=request),
                         **kwargs)
        
        self.script.define_environment_variable('MARS_ECFLOW_LABEL', self.info)
        self.script.define_environment_variable('MARS_TIMERS_FILE', "{}.timers".format(name))
        
        
class ArchiveTask(MarsTask):
    verb = 'archive'


class RetrieveTask(MarsTask):
    verb = 'retrieve'

There are two major object-oriented approaches to making encapsulated: inheritance and delegation.

### Suite Objects using Inheritance

In this first example we are going to choose to use *inheritance*, although this is a fairly arbitrary choice. Which is desirable depends very much on context. We are also going to avoid using the `ArchiveTask` defined above just to avoid having to put lots of safety-related code into these examples.

We wish to define a standard test pattern. This will:

1. Create a temporary (scratch) directory within the scratch space configured for the given host
2. Retrieve testing data, which is specified by the derived class
3. Run a test, which is defined by the derived class
4. Clean up after ourselves

In [4]:
class Cleanup(pf.Task):
    def __init__(self, path, name='cleanup', **kwargs):
        assert path != "/"
        super().__init__(name, script='rm -rf "{}"'.format(path), **kwargs)
      
    
class TestBase(pf.AnchorFamily):
    
    """This class is an interface"""
    
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)
        
        # Generate a unique working directory
        self._workdir = os.path.join(self.host.scratch_directory,
                                     self.suite.name, self.fullname.replace('/', '_'))
        
        # Ensure that the data gets put somewhere
        self._data_filename = 'retrieved.grib'
        request = self.request_dict().copy()
        request['target'] = self._data_filename
        
        with self:
            (
                RetrieveTask(request, workdir=self._workdir)
                >>
                self.build_test()
                >>
                Cleanup(self._workdir)
            )
            
    def request_dict(self):
        raise NotImplementedError("abstract base property")
        
    def build_test(self):
        raise NotImplementedError("abstract base method")

Classes should be derived from this abstract base test class, implementing the `request_dict` property and `build_test` methods. These derived classes can be further derived, or set up according to configuration passed in from outside.

In [5]:
class GribLsTest(TestBase):
    def __init__(self, date, param, **kwargs):
        self._date = date
        self._param = param
        name = kwargs.pop('name', 'grib_ls')
        super().__init__(name, **kwargs)
        
    def request_dict(self):
        return {
            'class': 'od',
            'expver': '0001',
            'stream': 'oper',
            'date': self._date,
            'time': [0, 12],
            'step': 0,
            'type': 'an',
            'levtype': 'ml',
            'levelist': 1,
            'param': self._param,
        }
    
    def build_test(self):
        return pf.Task('grib_ls', workdir=self._workdir, script='grib_ls -m {}'.format(self._data_filename))


class LsTest(TestBase):
    def __init__(self, **kwargs):
        super().__init__('ls', **kwargs)
        
    def request_dict(self):
        return {
            'class': 'od',
            'expver': '0001',
            'stream': 'oper',
            'date': -1,
            'time': [0, 12],
            'step': 0,
            'type': 'an',
            'levtype': 'ml',
            'levelist': 1,
            'param': 't',
        }
    
    def build_test(self):
        with pf.Family('test_family') as f:
            pf.Task('ls', workdir=self._workdir, script='ls -l {}'.format(self._data_filename))
        return f

These tests can be combined inside a suite.

In [6]:
with CourseSuite('inheritance_example') as s:
    with pf.Family('tests'):
        (
            GribLsTest(datetime.date.today() - datetime.timedelta(days=2), 't')
            >>
            GribLsTest(datetime.date.today() - datetime.timedelta(days=1), 'z', name='grib_ls_2')
            >>
            LsTest()
        )

s

CourseSuite(/inheritance_example)

### Suite Objects using Delegation

Alternatively, we can take the approach of delegation such that decisions about the data request to use and the test to construct are delegated to a configuration object that is injected from the controlling scope. If we do this then the resultant `Test` class is now a concrete class (and we no longer need to derive from it), changing the structure of the suite somewhat.

In this case, we build our `Test` class to delegate the construction to a config object whose type is unknown.

In [7]:
class DelegatingTest(pf.AnchorFamily):
    def __init__(self, config, **kwargs):
        
        name = config.name
        super().__init__(name, **kwargs)
        
        # Generate a unique working directory
        workdir = os.path.join(self.host.scratch_directory,
                               self.suite.name, self.fullname.replace('/', '_'))
        
        # Ensure that the data gets put somewhere
        data_filename = 'retrieved.grib'
        request = config.request_dict.copy()
        request['target'] = data_filename
        
        with self:
            (
                RetrieveTask(request, workdir=workdir)
                >>
                config.build_test(workdir, data_filename)
                >>
                Cleanup(workdir)
            )

We can now create config classes that provide this functionality. They do not have to be built in the same way, or related to each other in any way other than that they provide the given functionality.

In [8]:
class LsConfig:
    name = 'ls'
    request_dict = {
        'class': 'od',
        'expver': '0001',
        'stream': 'oper',
        'date': -1,
        'time': [0, 12],
        'step': 0,
        'type': 'an',
        'levtype': 'ml',
        'levelist': 1,
        'param': 't',
    }
    
    @staticmethod
    def build_test(workdir, data_filename):
        with pf.Family('test_family') as f:
            return pf.Task('ls', workdir=workdir, script='ls -l {}'.format(data_filename))


class GribLsConfig:
    
    def __init__(self, date, param, name='grib_ls'):
        self.name = name
        self._date = date
        self._param = param
        
    @property
    def request_dict(self):
        return {
            'class': 'od',
            'expver': '0001',
            'stream': 'oper',
            'date': self._date,
            'time': [0, 12],
            'step': 0,
            'type': 'an',
            'levtype': 'ml',
            'levelist': 1,
            'param': self._param,
        }
    
    def build_test(self, workdir, data_filename):
        return pf.Task('grib_ls', workdir=workdir, script='grib_ls -m {}'.format(data_filename))

We can then construct a combined configuration object.

In [9]:
class CombinedConfig:
    def __init__(self):
        self.tests = [
            GribLsConfig(datetime.date.today() - datetime.timedelta(days=2), 't'),
            GribLsConfig(datetime.date.today() - datetime.timedelta(days=1), 'z', name='grib_ls_2'),
            LsConfig # n.b. here we just used a raw class.
        ]

And we then configure the suite with the config object.

In [10]:
class DelegatedSuite(CourseSuite):
    def __init__(self, config):
        super().__init__('delegated_example')
        
        with self:
            pf.sequence(DelegatingTest(test_cfg) for test_cfg in config.tests)


s = DelegatedSuite(CombinedConfig())
s

DelegatedSuite(/delegated_example)

## Conditional Suite Structure

One of the goals of building an Object-Oriented suite is avoiding tangled, procedural complexity in constructing suites. Making a suite configurable, and multi-purpose requires conditionality in how the suite is constructed.

The most obvious way to do this is to put conditional expressions, namely if statements, into the suite structure. This works, but leads to a long-term increase in the complexity of the suite. But worse, it puts the configuration- and system-dependent logic about how a suite should be built into the structure of the suite rather than with the configuration where it belongs.

This example shows delegation of conditional behaviour to a configuration, such that the configuration can use arbitrary logic and complexity (in this case just a lookup) to determine which subsections of a suite get built.

In [11]:
class Config:
    
    def __init__(self, **tests):
        
        # Default tests that should be built. Otherwise assume not
        self.enabled_tests = {
            'test3': True
        }
        self.enabled_tests.update(tests)
        
    def build_test(self, cls, name, *args, **kwargs):
        if self.enabled_tests.get(name, False):
            return cls(name, *args, **kwargs)
        
        
class ATest(pf.Task):
    def __init__(self, name, val):
        super().__init__(name, script="echo test={} : val={}".format(name, val))
        

class TestingSuite(CourseSuite):
    
    def __init__(self, name, config, **kwargs):
        super().__init__(name, **kwargs)
        with self:
            config.build_test(ATest, 'test1', 1234)
            config.build_test(ATest, 'test2', 4321)
            config.build_test(ATest, 'test3', 6666)
            config.build_test(ATest, 'test4', 7777)


TestingSuite('default_tests', Config())

TestingSuite(/default_tests)

In [12]:
TestingSuite('add_test4', Config(test4=True))

TestingSuite(/add_test4)

In [13]:
TestingSuite('override_default_test', Config(test1=True, test2=True, test3=False))

TestingSuite(/override_default_test)

### Structural Delegation

This first example demonstrates delegating a structural decision to a configuration object. We wish to loop over two different axes - one an integer axis, and the other a string based one. The configuration objects decide how this should be done, and the order of the looping.

Further configuration objects can be derived from `Config1` and `Config2` to update the values, while leaving the structures the same.

Once the suite has delegated construction of the looping structure to the config, the construction of the tasks within the looping structure can be continued in the normal way.

In [14]:
class ConfigBase:
    suite_name = None
    min_integer = 1
    max_integer = 5
    strings = ['a', 'b', 'c', 'd', 'e']
    
    def build_nested_loops(self, **kwargs):
        raise NotImplementedError


class Config1(ConfigBase):
    suite_name = 'config_string_integer'
    def build_nested_loops(self, **kwargs):
        with pf.Family('string_looper'):
            pf.RepeatEnumerated('REPEAT_STRING', self.strings)
            with pf.Family('integer_looper', **kwargs) as inner:
                pf.RepeatInteger('REPEAT_INTEGER', self.min_integer, self.max_integer)
        return inner


class Config2(ConfigBase):
    suite_name = 'config_integer_string'
    def build_nested_loops(self, **kwargs):
        with pf.Family('integer_looper'):
            pf.RepeatInteger('REPEAT_INTEGER', self.min_integer, self.max_integer)
            with pf.Family('string_looper', **kwargs) as inner:
                pf.RepeatEnumerated('REPEAT_STRING', self.strings)
        return inner


class NestedLoopingSuite(CourseSuite):
    def __init__(self, config):
        super().__init__(config.suite_name)
        
        with self:
            with config.build_nested_loops(labels={'info': ''}) as f:
                (                
                    LabelSetter((f.info, '$REPEAT_INTEGER : $REPEAT_STRING'))
                    >>
                    WaitSeconds(2)
                )


s1 = NestedLoopingSuite(Config1())
s2 = NestedLoopingSuite(Config2())
s1

NestedLoopingSuite(/config_string_integer)

In [15]:
s1.deploy_suite(pf.Notebook)

In [16]:
s2

NestedLoopingSuite(/config_integer_string)

In [17]:
s2.deploy_suite(pf.Notebook)